<a href="https://colab.research.google.com/github/yelynn99/study-2020-2/blob/master/%EC%8B%A0%EA%B2%BD%EB%A7%9D%20%EC%98%A4%EB%A5%98%EC%9E%88%EB%8A%94%EA%B2%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_openml
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# loading the dataset using fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

X, y= mnist["data"], mnist["target"]
print(X.shape, y.shape)
y=y.reshape(-1,1)
print(y.shape)
y = y.astype(np.uint8)
y
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
training_data=np.hstack((y_train,X_train))
test_data=np.hstack((y_test,X_test))
print(training_data.shape,test_data.shape)
print(training_data)

In [ ]:
#sigmoid 함수
def sigmoid(x):
    return 1/ (1+np.exp(-x))
    
#수치미분 함수
def numerical_derivative(f,x):
    delta_x = 1e-4
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx=it.multi_index
        tmp_val = x[idx]
        
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x)
        
        x[idx] = float(tmp_val) - delta_x
        fx2 = f(x)
        
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val
        it.iternext()
        
        return grad

In [ ]:
class NeuralNetwork:
    def __init__(self, input_nodes, hidden_nodes, output_nodes):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        
        self.w2 = np.random.rand(self.input_nodes, self.hidden_nodes)
        self.b2 = np.random.rand(self.hidden_nodes)
        
        self.w3 = np.random.rand(self.hidden_nodes, output_nodes)
        self.b3 = np.random.rand(self.output_nodes)
        
        self.learning_rate = 1e-4
        
    def feed_forward(self):
        delta = 1e-7
        
        z1 = np.dot(self.input_data, self.w2) + self.b2
        a1 = sigmoid(z1)
        
        z2 = np.dot(a1, self.w3) + self.b3
        y = sigmoid(z2)
        
        return -np.sum(self.target_data*np.log(y+delta)+(1-self.target_data)*np.log((1-y)+delta)) #cross_entropy loss ftn
        
    def train(self, training_data):
        #normalize
        self.target_data = np.zeros(output_nodes) + 0.01
        self.target_data[int(training_data[0])] = 0.99
        self.input_data = (training_data[1:]/255.0*0.99) + 0.01
        
        f = lambda x: self.feed_forward()
        
        self.w2 -= self.learning_rate * numerical_derivative(f, self.w2)
        self.b2 -= self.learning_rate * numerical_derivative(f, self.b2)
        self.w3 -= self.learning_rate * numerical_derivative(f, self.w3)
        self.b3 -= self.learning_rate * numerical_derivative(f, self.b3)
        
    def predict(self, input_data):
        z1 = np.dot(input_data, self.w2) + self.b2
        y1 = sigmoid(z1)
        z2 = np.dot(y1, self.w3) + self.b3
        y = sigmoid(z2)
        predicted_num = np.argmax(y)
        
        return predicted_num
        
    def accuracy(self, test_data):
        matched_list = []
        not_matched_list = []
        
        for index in range(len(test_data)):
            label = int(test_data[index, 0])
            
            data = (test_data[index, 1:]/255.0*0.99)+ 0.01
            
            predicted_num = self.predict(data)
            
            if label == predicted_num:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        print('current accuracy=', 100*(len(matched_list)/(len(test_data))), "%")
        
        return matched_list, not_matched_list

In [ ]:
input_nodes = 784
hidden_nodes = 100
output_nodes = 10

nn = NeuralNetwork(input_nodes, hidden_nodes, output_nodes)

for step in range(30001):
    index = np.random.randint(0,len(training_data)-1)
    nn.train(training_data[index])
    
    if step %400 == 0:
        print("step=",step," loss_val=",nn.feed_forward())
        
     
nn.accuracy(test_data)